# Building an Agentic Stock Analyzer: LangGraph + Amazon SageMaker Jumpstart + Amazon Bedrock AgentCore

## Overview

In this tutorial, we will learn how to build and deploy an intelligent stock analysis agent using Amazon Bedrock 
AgentCore Runtime with LangGraph and Amazon SageMaker.

We will focus on creating a complete stock investment workflow that combines:
• **LangGraph** for multi-step agent orchestration
• **Amazon SageMaker** with OpenAI GPT-OSS models for natural language processing
• **Custom stock analysis tools** for data gathering, performance analysis, and investment decisions
• **Amazon Bedrock AgentCore Runtime** for scalable cloud deployment
• **Amazon S3** for automated PDF report generation and storage

This example demonstrates how to transform simple stock ticker requests into comprehensive investment analysis with 
automated buy/sell/hold recommendations, complete with risk assessments and professional PDF reports.

For other agent frameworks and model combinations, check out:
• [Strands Agents with Amazon Bedrock models](../01-strands-with-bedrock-model)
• [Strands Agents with OpenAI models](../03-strands-with-openai-model)

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | LangGraph                                                                    |
| LLM model           | Sagemaker Jumpstart GPT OSS 20B                                                 |
| Tutorial components | Hosting agent on AgentCore Runtime. Using LangGraph, Sagemaker Jumpstart Model, and S3 |
| Tutorial vertical   | Financial Services / Investment Analysis                                     |
| Example complexity  | Intermediate                                                                 |
| SDK used            | Amazon BedrockAgentCore Python SDK, boto3, yfinance                         |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime.

For demonstration purposes, we will use a LangGraph agent using Amazon SageMaker with OpenAI GPT-OSS models for 
intelligent stock analysis and investment recommendations.

In our example we will use a sophisticated stock analysis agent with three specialized tools:  

• gather_stock_data - Collects real-time market data, financial metrics, and company information  
• analyze_stock_performance - Performs comprehensive technical and fundamental analysis with risk assessment    
• The generate_stock_report tool creates professional PDF reports from the gathered stock data and analysis, automatically uploading them to Amazon S3 with organized date-based folders. 

This agent demonstrates a complete end-to-end workflow that transforms simple stock symbol requests into professional investment analysis with calculated risk levels, position sizing recommendations, and automated PDF report storage in Amazon S3.

### Tutorial Key Features  
  
• Hosting Agents on Amazon Bedrock AgentCore Runtime  
• Using Amazon Sagemaker AI models, especially GPT OSS 20B model  
• Using LangGraph for multi-agent orchestration  
• Real-time stock data integration with yfinance API  
• Automated PDF report generation with ReportLab  
• Amazon S3 integration for document storage  
• Comprehensive investment analysis with technical and fundamental metrics  
• Risk-adjusted portfolio recommendations  

### What You'll Build

By the end of this tutorial, you'll have a production-ready stock analysis agent that can:

1. Accept natural language requests like "Analyze AAPL stock for investment"
2. Gather comprehensive market data including current prices, financial ratios, and historical performance
3. Perform professional analysis using both technical indicators and fundamental metrics
4. Create detailed PDF reports with executive summaries, risk assessments, and monitoring recommendations
5. Store reports automatically in Amazon S3 with organized date-based folder structure
6. Scale seamlessly using Amazon Bedrock AgentCore Runtime infrastructure

This intelligent agent combines the power of real-time market data, advanced AI analysis, and professional report 
generation to provide institutional-quality investment research at scale.


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt

## Prerequisites: Deploy GPT-OSS Model on Amazon SageMaker

Before we begin building our stock analyzer agent, you'll need to deploy the OpenAI GPT-OSS model using Amazon SageMaker JumpStart.

**Required Setup:**
1. Open the notebook `openai_gpt_oss.ipynb` in the `./deploy_sagemaker/gpt-oss` folder
2. Follow the instructions to deploy the GPT-OSS model to a SageMaker endpoint
3. Note down the **endpoint name** that gets created (you'll need this for our agent configuration)
4. Return to this notebook once your SageMaker endpoint is successfully deployed

The SageMaker endpoint will serve as the language model backend for our intelligent stock analyzer agent.

In [ ]:
sagemaker_endpoint_name = "gpt-oss-120b-2025-08-26-23-50-17-101"
assert sagemaker_endpoint_name != ""

bucket_name = "surya-495365983931"
assert bucket_name != ""

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.



In [ ]:
def create_local_stock_agent_file(endpoint_name, filename="langgraph_stock_local.py"):
    """
    Create a local stock analysis agent file with the specified SageMaker endpoint name
    
    Args:
        endpoint_name: SageMaker endpoint name to use
        filename: Output filename (default: langgraph_stock_local.py)
    """
    
    code_content = f'''from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
import argparse
import json
import re
import requests
import yfinance as yf
from datetime import datetime, timedelta
from typing import Dict
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import os

# Create stock analysis tools
@tool
def gather_stock_data(stock_symbol: str) -> str:
    """
    Gather comprehensive stock data from various sources including price history, 
    financial metrics, news, and market data.
    
    Args:
        stock_symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'TSLA')
    
    Returns:
        Comprehensive stock data including current price, historical performance, 
        financial metrics, and recent news
    """
    try:
        # Clean the stock symbol
        symbol = stock_symbol.upper().strip()
        
        # Get stock data using yfinance
        stock = yf.Ticker(symbol)
        
        # Get basic info
        info = stock.info
        
        # Get historical data (1 year)
        hist = stock.history(period="1y")
        current_price = hist['Close'].iloc[-1] if not hist.empty else 0
        
        # Calculate performance metrics
        if len(hist) > 0:
            year_high = hist['High'].max()
            year_low = hist['Low'].min()
            year_start_price = hist['Close'].iloc[0]
            ytd_return = ((current_price - year_start_price) / year_start_price) * 100
            
            # Calculate volatility (standard deviation of daily returns)
            daily_returns = hist['Close'].pct_change().dropna()
            volatility = daily_returns.std() * (252 ** 0.5) * 100  # Annualized volatility
        else:
            year_high = year_low = ytd_return = volatility = 0
            
        # Get recent news (simulated - in production you'd use a real news API)
        recent_news = [
            f"{{symbol}} reports quarterly earnings with mixed results",
            f"Analysts upgrade {{symbol}} price target amid strong fundamentals",
            f"{{symbol}} announces new strategic partnership",
            f"Market volatility affects {{symbol}} trading volume"
        ]
        
        # Format the comprehensive data
        stock_data = f"""STOCK DATA GATHERING REPORT:
================================
Stock Symbol: {{symbol}}
Company Name: {{info.get('longName', 'N/A')}}
Sector: {{info.get('sector', 'N/A')}}
Industry: {{info.get('industry', 'N/A')}}

CURRENT MARKET DATA:
- Current Price: ${{current_price:.2f}}
- Market Cap: ${{info.get('marketCap', 0):,}} 
- 52-Week High: ${{year_high:.2f}}
- 52-Week Low: ${{year_low:.2f}}
- YTD Return: {{ytd_return:.2f}}%
- Volatility (Annualized): {{volatility:.2f}}%

FINANCIAL METRICS:
- P/E Ratio: {{info.get('trailingPE', 'N/A')}}
- Forward P/E: {{info.get('forwardPE', 'N/A')}}
- Price-to-Book: {{info.get('priceToBook', 'N/A')}}
- Dividend Yield: {{info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0:.2f}}%
- Revenue (TTM): ${{info.get('totalRevenue', 0):,}}
- Profit Margin: {{info.get('profitMargins', 0) * 100 if info.get('profitMargins') else 0:.2f}}%

TRADING METRICS:
- Average Volume: {{info.get('averageVolume', 0):,}}
- Beta: {{info.get('beta', 'N/A')}}
- EPS (TTM): ${{info.get('trailingEps', 'N/A')}}
- Book Value: ${{info.get('bookValue', 'N/A')}}

RECENT NEWS HEADLINES:
{{chr(10).join(f"- {{news}}" for news in recent_news)}}

DATA COLLECTION TIMESTAMP: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}
"""
        
        return stock_data
        
    except Exception as e:
        return f"""STOCK DATA GATHERING ERROR:
================================
Stock Symbol: {{stock_symbol}}
Error: Unable to gather comprehensive stock data
Details: {{str(e)}}

Please verify the stock symbol is correct and try again.
"""

@tool
def analyze_stock_performance(stock_data: str) -> str:
    """
    Analyze stock performance based on gathered data, providing technical analysis,
    fundamental analysis, and risk assessment WITHOUT investment recommendations.
    
    Args:
        stock_data: Raw stock data from the data gathering agent
    
    Returns:
        Comprehensive stock analysis including technical indicators, fundamental analysis,
        and risk assessment for informational purposes only
    """
    import re
    
    # Extract key metrics from stock data
    symbol_match = re.search(r'Stock Symbol: ([A-Z]+)', stock_data)
    price_match = re.search(r'Current Price: \\$([\\d.]+)', stock_data)
    pe_match = re.search(r'P/E Ratio: ([\\d.]+)', stock_data)
    ytd_match = re.search(r'YTD Return: ([\\d.-]+)%', stock_data)
    volatility_match = re.search(r'Volatility \\(Annualized\\): ([\\d.]+)%', stock_data)
    dividend_match = re.search(r'Dividend Yield: ([\\d.]+)%', stock_data)
    beta_match = re.search(r'Beta: ([\\d.]+)', stock_data)
    profit_margin_match = re.search(r'Profit Margin: ([\\d.]+)%', stock_data)
    
    symbol = symbol_match.group(1) if symbol_match else 'UNKNOWN'
    current_price = float(price_match.group(1)) if price_match else 0
    pe_ratio = float(pe_match.group(1)) if pe_match and pe_match.group(1) != 'N/A' else None
    ytd_return = float(ytd_match.group(1)) if ytd_match else 0
    volatility = float(volatility_match.group(1)) if volatility_match else 0
    dividend_yield = float(dividend_match.group(1)) if dividend_match else 0
    beta = float(beta_match.group(1)) if beta_match and beta_match.group(1) != 'N/A' else None
    profit_margin = float(profit_margin_match.group(1)) if profit_margin_match else 0
    
    # Technical Analysis (descriptive only)
    if ytd_return > 20:
        price_trend = "STRONG UPTREND"
    elif ytd_return > 10:
        price_trend = "MODERATE UPTREND"
    elif ytd_return > 0:
        price_trend = "SLIGHT UPTREND"
    elif ytd_return > -10:
        price_trend = "SLIGHT DOWNTREND"
    else:
        price_trend = "STRONG DOWNTREND"
    
    # Fundamental Analysis (descriptive only)
    fundamental_factors = []
    
    if pe_ratio:
        if pe_ratio < 15:
            fundamental_factors.append("P/E ratio suggests potential undervaluation")
        elif pe_ratio < 25:
            fundamental_factors.append("P/E ratio within reasonable range")
        else:
            fundamental_factors.append("P/E ratio suggests potential overvaluation")
    
    if profit_margin > 20:
        fundamental_factors.append("Excellent profit margins")
    elif profit_margin > 10:
        fundamental_factors.append("Good profit margins")
    else:
        fundamental_factors.append("Low profit margins")
    
    if dividend_yield > 3:
        fundamental_factors.append("High dividend yield")
    elif dividend_yield > 1:
        fundamental_factors.append("Moderate dividend yield")
    else:
        fundamental_factors.append("Low or no dividend yield")
    
    beta_description = ""
    if beta and beta > 1.5:
        beta_description = "High beta indicates sensitivity to market movements"
    elif beta and beta < 0.5:
        beta_description = "Low beta indicates stability relative to market"
    else:
        beta_description = "Beta indicates moderate market correlation"
    
    analysis_report = f"""STOCK PERFORMANCE ANALYSIS:
===============================
Stock: {{symbol}} | Current Price: ${{current_price:.2f}}

TECHNICAL ANALYSIS:
- Price Trend: {{price_trend}}
- YTD Performance: {{ytd_return:.2f}}%
- Volatility Level: {{volatility:.2f}}% ({{risk_level}} RISK)
- Volatility Assessment: {{risk_description}}

FUNDAMENTAL ANALYSIS:
- P/E Ratio: {{pe_ratio if pe_ratio else 'N/A'}}
- Profit Margin: {{profit_margin:.2f}}%
- Dividend Yield: {{dividend_yield:.2f}}%
- Beta: {{beta if beta else 'N/A'}}

KEY OBSERVATIONS:
{{chr(10).join(f"• {{factor}}" for factor in fundamental_factors)}}


ANALYST SUMMARY:
Based on technical and fundamental analysis, {{symbol}} shows {{price_trend.lower()}} with {{risk_level.lower()}} volatility profile. 
The analysis reflects current market conditions and financial performance metrics for informational purposes.

DISCLAIMER: This analysis is for informational purposes only and does not constitute investment advice.
"""
    
    return analysis_report

@tool
def generate_stock_report(stock_data: str, analysis_data: str) -> str:
    """
    Generate a comprehensive stock report based on gathered data and analysis.
    Creates a professional PDF report for documentation purposes.
    
    Args:
        stock_data: Raw stock data from the data gathering agent
        analysis_data: Analysis results from the performance analyzer
    
    Returns:
        Report generation summary with PDF creation status
    """
    import re
    
    # Extract key information for report
    symbol_match = re.search(r'Stock Symbol: ([A-Z]+)', stock_data)
    price_match = re.search(r'Current Price: \\$([\\d.]+)', stock_data)
    company_match = re.search(r'Company Name: ([^\\n]+)', stock_data)
    sector_match = re.search(r'Sector: ([^\\n]+)', stock_data)
    ytd_match = re.search(r'YTD Performance: ([\\d.-]+)%', analysis_data)
    risk_match = re.search(r'Volatility Risk: ([A-Z]+)', analysis_data)
    
    symbol = symbol_match.group(1) if symbol_match else 'UNKNOWN'
    current_price = float(price_match.group(1)) if price_match else 0
    company_name = company_match.group(1).strip() if company_match else 'N/A'
    sector = sector_match.group(1).strip() if sector_match else 'N/A'
    ytd_performance = float(ytd_match.group(1)) if ytd_match else 0
    risk_level = risk_match.group(1) if risk_match else 'MEDIUM'
    
    # Generate PDF report filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_filename = f"{{symbol}}_Stock_Report_{{timestamp}}.pdf"
    
    # Create PDF report
    try:
        create_stock_report_pdf(symbol, company_name, sector, current_price, 
                              ytd_performance, risk_level, stock_data, analysis_data, pdf_filename)
        pdf_status = f"PDF report generated: {{pdf_filename}}"
    except Exception as e:
        pdf_status = f"PDF generation failed: {{str(e)}}"
    
    report_summary = f"""STOCK REPORT GENERATION:
===============================
Stock: {{symbol}} ({{company_name}})
Sector: {{sector}}
Current Price: ${{current_price:.2f}}

REPORT SUMMARY:
- Technical Analysis: {{ytd_performance:.2f}}% YTD performance
- Risk Assessment: {{risk_level}} volatility risk
- Report Type: Comprehensive stock analysis for informational purposes
- Generated: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}

{{pdf_status}}

REPORT CONTENTS:
• Executive Summary with key metrics
• Detailed market data and financial metrics
• Technical and fundamental analysis
• Risk assessment and observations
• Professional formatting for documentation

DISCLAIMER: This report is for informational and educational purposes only. 
It does not constitute investment advice or recommendations.
"""
    
    return report_summary

def create_stock_report_pdf(symbol, company_name, sector, price, ytd_perf, risk_level, stock_data, analysis_data, filename):
    """Create a professional PDF stock report"""
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []
    
    # Title
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=18,
        spaceAfter=30,
        textColor=colors.darkblue
    )
    story.append(Paragraph(f"Stock Analysis Report: {{symbol}}", title_style))
    story.append(Spacer(1, 12))
    
    # Executive Summary
    story.append(Paragraph("Executive Summary", styles['Heading2']))
    summary_data = [
        ['Metric', 'Value'],
        ['Stock Symbol', symbol],
        ['Company Name', company_name],
        ['Sector', sector],
        ['Current Price', f"${{price:.2f}}"],
        ['YTD Performance', f"{{ytd_perf:.2f}}%"],
        ['Risk Level', risk_level]
    ]
    
    summary_table = Table(summary_data)
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    story.append(summary_table)
    story.append(Spacer(1, 20))
    
    # Stock Data Section
    story.append(Paragraph("Market Data", styles['Heading2']))
    story.append(Paragraph(stock_data.replace('\\n', '<br/>'), styles['Normal']))
    story.append(Spacer(1, 20))
    
    # Analysis Section
    story.append(Paragraph("Performance Analysis", styles['Heading2']))
    story.append(Paragraph(analysis_data.replace('\\n', '<br/>'), styles['Normal']))
    
    # Generate timestamp
    story.append(Spacer(1, 20))
    story.append(Paragraph(f"Report Generated: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}", styles['Normal']))
    story.append(Paragraph("This report is for informational purposes only.", styles['Normal']))
    
    doc.build(story)

# Custom wrapper to make SagemakerEndpoint work with LangGraph tool binding
class SagemakerLLMWrapper:
    def __init__(self, sagemaker_llm, tools):
        self.sagemaker_llm = sagemaker_llm
        self.tools = tools
    
    def bind_tools(self, tools):
        self.tools = tools
        return self
    
    def invoke(self, messages):
        # Extract the user message content
        user_content = ""
        for msg in messages:
            if isinstance(msg, HumanMessage):
                user_content = msg.content
                break
        
        # Check if this is a stock analysis request
        if any(keyword in user_content.lower() for keyword in ['analyze', 'stock', 'ticker', 'symbol']):
            # Extract stock symbol from user input
            stock_match = re.search(r'\\b([A-Z]{{2,5}})\\b', user_content.upper())
            if stock_match:
                stock_symbol = stock_match.group(1)
                
                # Step 1: Gather stock data
                print(f"Step 1: Gathering data for {{stock_symbol}}...")
                stock_data = self.tools[0].invoke({{"stock_symbol": stock_symbol}})
                
                # Step 2: Analyze stock performance
                print(f"Step 2: Analyzing {{stock_symbol}} performance...")
                analysis_result = self.tools[1].invoke({{"stock_data": stock_data}})
                
                # Step 3: Generate stock report
                print(f"Step 3: Generating report for {{stock_symbol}}...")
                report_result = self.tools[2].invoke({{"stock_data": stock_data, "analysis_data": analysis_result}})
                
                # Return comprehensive response
                full_response = f"""**COMPREHENSIVE STOCK ANALYSIS REPORT**

**Step 1 - Stock Data Gathering:**
{{stock_data}}

**Step 2 - Performance Analysis:**
{{analysis_result}}

**Step 3 - Report Generation:**
{{report_result}}

---
**ANALYSIS COMPLETE:** Comprehensive stock analysis has been performed and a detailed PDF report has been generated for documentation purposes."""
                
                return AIMessage(content=full_response)
            else:
                return AIMessage(content="Please provide a valid stock symbol (e.g., AAPL, GOOGL, TSLA) for analysis.")
        
        # For other messages, use the SageMaker model normally
        system_msg = """You are a professional stock analyst. Provide helpful responses about stock analysis, market trends, and financial metrics for informational purposes only."""
        
        full_prompt = f"{{system_msg}}\\n\\nUser: {{user_content}}"
        
        # Get response from SageMaker endpoint
        response = self.sagemaker_llm.invoke(full_prompt)
        
        # Return a proper LangChain AIMessage
        return AIMessage(content=response)

# Define the agent using SageMaker endpoint
def create_agent():
    """Create and configure the LangGraph stock analysis agent with SageMaker endpoint"""
    
    # Your SageMaker endpoint configuration
    endpoint_name = "{endpoint_name}"
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            # GPT-OSS harmony format payload structure
            payload = {{
                "model": "/opt/ml/model",
                "input": [
                    {{
                        "role": "system",
                        "content": "You are a professional stock analyst. Analyze stocks and provide detailed information for educational purposes only, without investment recommendations."
                    }},
                    {{
                        "role": "user",
                        "content": prompt
                    }}
                ],
                "max_output_tokens": model_kwargs.get("max_new_tokens", 2048),
                "stream": "false",
                "temperature": model_kwargs.get("temperature", 0.1),
                "top_p": model_kwargs.get("top_p", 1)
            }}
            input_str = json.dumps(payload)
            return input_str.encode("utf-8")

        def transform_output(self, output: bytes) -> str:
            # Parse harmony format response
            decoded_output = output.read().decode("utf-8")
            response_json = json.loads(decoded_output)
            
            if 'output' in response_json and isinstance(response_json['output'], list):
                for item in response_json['output']:
                    if item.get('type') == 'message' and item.get('role') == 'assistant':
                        content = item.get('content', [])
                        for content_item in content:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
                
                # Fallback parsing for different harmony format structures
                for item in response_json['output']:
                    if item.get('type') != 'reasoning' and 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
            
            # Final fallback - return raw response
            return str(response_json)
    
    # Initialize SageMaker LLM with harmony format
    content_handler = ContentHandler()
    sagemaker_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-2",
        model_kwargs={{
            "max_new_tokens": 2048, 
            "do_sample": True, 
            "temperature": 0.1,  # Lower temperature for consistent analysis
            "top_p": 1
        }},
        content_handler=content_handler
    )
    
    # Create tools (3 tools: data gathering, analysis, report generation)
    tools = [gather_stock_data, analyze_stock_performance, generate_stock_report]
    
    # Wrap SageMaker LLM to work with LangGraph
    llm_with_tools = SagemakerLLMWrapper(sagemaker_llm, tools)
    
    # System message for stock analysis
    system_message = """You are a professional stock analyst with expertise in technical analysis, fundamental analysis, and report generation. 

Your role is to:
1. Gather comprehensive stock data from multiple sources including price history, financial metrics, and market data
2. Analyze stock performance using both technical and fundamental analysis techniques
3. Generate professional stock reports for documentation and educational purposes

Provide informational analysis only, without investment recommendations or advice."""
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {{"messages": [response]}}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_stock_sagemaker(payload):
    """
    Invoke the stock analysis agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({{"messages": [HumanMessage(content=user_input)]}})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_stock_sagemaker(json.loads(args.payload))
    print(response)
'''
    
    # Write the file
    with open(filename, 'w') as f:
        f.write(code_content)
    
    print(f"Created {filename} with endpoint: {endpoint_name}")
    return filename

# Example usage
if __name__ == "__main__":
    # Replace with your actual SageMaker endpoint name
    create_local_stock_agent_file(sagemaker_endpoint_name)

#### Invoking local agent

In [ ]:
!python langgraph_stock_local.py '{"prompt": "Analyze AAPL stock for investment"}'

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### LangGraph with Amazon SageMaker JumpStart GPT-OSS model
Let's start with our LangGraph using Amazon SageMaker JumpStart GPT-OSS 20B model. Other examples with different 
frameworks and models are available in the parent directories

In [ ]:
def create_agentcore_deployment_file(endpoint_name, bucket_name, filename="langgraph_stock_sagemaker_gpt_oss.py"):
    """
    Create an AgentCore deployment file with the specified SageMaker endpoint name
    
    Args:
        endpoint_name: SageMaker endpoint name to use
        bucket_name: S3 bucket name to use
        filename: Output filename (default: langgraph_stock_sagemaker_gpt_oss.py)
    """
    
    code_content = f'''from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import re
import yfinance as yf
from datetime import datetime, timedelta
from typing import Dict
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import boto3
import os
import tempfile

app = BedrockAgentCoreApp()

# Initialize S3 client
s3_client = boto3.client('s3')
S3_BUCKET_NAME = "{bucket_name}"

# Create stock analysis tools
@tool
def gather_stock_data(stock_symbol: str) -> str:
    """
    Gather comprehensive stock data from various sources including price history, 
    financial metrics, news, and market data.
    
    Args:
        stock_symbol: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'TSLA')
    
    Returns:
        Comprehensive stock data including current price, historical performance, 
        financial metrics, and recent news
    """
    try:
        # Clean the stock symbol
        symbol = stock_symbol.upper().strip()
        
        # Get stock data using yfinance
        stock = yf.Ticker(symbol)
        
        # Get basic info
        info = stock.info
        
        # Get historical data (1 year)
        hist = stock.history(period="1y")
        current_price = hist['Close'].iloc[-1] if not hist.empty else 0
        
        # Calculate performance metrics
        if len(hist) > 0:
            year_high = hist['High'].max()
            year_low = hist['Low'].min()
            year_start_price = hist['Close'].iloc[0]
            ytd_return = ((current_price - year_start_price) / year_start_price) * 100
            
            # Calculate volatility (standard deviation of daily returns)
            daily_returns = hist['Close'].pct_change().dropna()
            volatility = daily_returns.std() * (252 ** 0.5) * 100  # Annualized volatility
        else:
            year_high = year_low = ytd_return = volatility = 0
            
        # Get recent news (simulated - in production you'd use a real news API)
        recent_news = [
            f"{{symbol}} reports quarterly earnings with mixed results",
            f"Analysts upgrade {{symbol}} price target amid strong fundamentals",
            f"{{symbol}} announces new strategic partnership",
            f"Market volatility affects {{symbol}} trading volume"
        ]
        
        # Format the comprehensive data
        stock_data = f"""STOCK DATA GATHERING REPORT:
================================
Stock Symbol: {{symbol}}
Company Name: {{info.get('longName', 'N/A')}}
Sector: {{info.get('sector', 'N/A')}}
Industry: {{info.get('industry', 'N/A')}}

CURRENT MARKET DATA:
- Current Price: ${{current_price:.2f}}
- Market Cap: ${{info.get('marketCap', 0):,}} 
- 52-Week High: ${{year_high:.2f}}
- 52-Week Low: ${{year_low:.2f}}
- YTD Return: {{ytd_return:.2f}}%
- Volatility (Annualized): {{volatility:.2f}}%

FINANCIAL METRICS:
- P/E Ratio: {{info.get('trailingPE', 'N/A')}}
- Forward P/E: {{info.get('forwardPE', 'N/A')}}
- Price-to-Book: {{info.get('priceToBook', 'N/A')}}
- Dividend Yield: {{info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0:.2f}}%
- Revenue (TTM): ${{info.get('totalRevenue', 0):,}}
- Profit Margin: {{info.get('profitMargins', 0) * 100 if info.get('profitMargins') else 0:.2f}}%

TRADING METRICS:
- Average Volume: {{info.get('averageVolume', 0):,}}
- Beta: {{info.get('beta', 'N/A')}}
- EPS (TTM): ${{info.get('trailingEps', 'N/A')}}
- Book Value: ${{info.get('bookValue', 'N/A')}}

RECENT NEWS HEADLINES:
{{chr(10).join(f"- {{news}}" for news in recent_news)}}

DATA COLLECTION TIMESTAMP: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}
"""
        
        return stock_data
        
    except Exception as e:
        return f"""STOCK DATA GATHERING ERROR:
================================
Stock Symbol: {{stock_symbol}}
Error: Unable to gather comprehensive stock data
Details: {{str(e)}}

Please verify the stock symbol is correct and try again.
"""

@tool
def analyze_stock_performance(stock_data: str) -> str:
    """
    Analyze stock performance based on gathered data, providing technical analysis,
    fundamental analysis, and risk assessment WITHOUT investment recommendations.
    
    Args:
        stock_data: Raw stock data from the data gathering agent
    
    Returns:
        Comprehensive stock analysis including technical indicators, fundamental analysis,
        and risk assessment for informational purposes only
    """
    import re
    
    # Extract key metrics from stock data
    symbol_match = re.search(r'Stock Symbol: ([A-Z]+)', stock_data)
    price_match = re.search(r'Current Price: \\$([\\d.]+)', stock_data)
    pe_match = re.search(r'P/E Ratio: ([\\d.]+)', stock_data)
    ytd_match = re.search(r'YTD Return: ([\\d.-]+)%', stock_data)
    volatility_match = re.search(r'Volatility \\(Annualized\\): ([\\d.]+)%', stock_data)
    dividend_match = re.search(r'Dividend Yield: ([\\d.]+)%', stock_data)
    beta_match = re.search(r'Beta: ([\\d.]+)', stock_data)
    profit_margin_match = re.search(r'Profit Margin: ([\\d.]+)%', stock_data)
    
    symbol = symbol_match.group(1) if symbol_match else 'UNKNOWN'
    current_price = float(price_match.group(1)) if price_match else 0
    pe_ratio = float(pe_match.group(1)) if pe_match and pe_match.group(1) != 'N/A' else None
    ytd_return = float(ytd_match.group(1)) if ytd_match else 0
    volatility = float(volatility_match.group(1)) if volatility_match else 0
    dividend_yield = float(dividend_match.group(1)) if dividend_match else 0
    beta = float(beta_match.group(1)) if beta_match and beta_match.group(1) != 'N/A' else None
    profit_margin = float(profit_margin_match.group(1)) if profit_margin_match else 0
    
    # Technical Analysis (descriptive only)
    if ytd_return > 20:
        price_trend = "STRONG UPTREND"
    elif ytd_return > 10:
        price_trend = "MODERATE UPTREND"
    elif ytd_return > 0:
        price_trend = "SLIGHT UPTREND"
    elif ytd_return > -10:
        price_trend = "SLIGHT DOWNTREND"
    else:
        price_trend = "STRONG DOWNTREND"
    
    # Fundamental Analysis (descriptive only)
    fundamental_factors = []
    
    if pe_ratio:
        if pe_ratio < 15:
            fundamental_factors.append("P/E ratio suggests potential undervaluation")
        elif pe_ratio < 25:
            fundamental_factors.append("P/E ratio within reasonable range")
        else:
            fundamental_factors.append("P/E ratio suggests potential overvaluation")
    
    if profit_margin > 20:
        fundamental_factors.append("Excellent profit margins")
    elif profit_margin > 10:
        fundamental_factors.append("Good profit margins")
    else:
        fundamental_factors.append("Low profit margins")
    
    if dividend_yield > 3:
        fundamental_factors.append("High dividend yield")
    elif dividend_yield > 1:
        fundamental_factors.append("Moderate dividend yield")
    else:
        fundamental_factors.append("Low or no dividend yield")
    
    
    beta_description = ""
    if beta and beta > 1.5:
        beta_description = "High beta indicates sensitivity to market movements"
    elif beta and beta < 0.5:
        beta_description = "Low beta indicates stability relative to market"
    else:
        beta_description = "Beta indicates moderate market correlation"
    
    analysis_report = f"""STOCK PERFORMANCE ANALYSIS:
===============================
Stock: {{symbol}} | Current Price: ${{current_price:.2f}}

TECHNICAL ANALYSIS:
- Price Trend: {{price_trend}}
- YTD Performance: {{ytd_return:.2f}}%


FUNDAMENTAL ANALYSIS:
- P/E Ratio: {{pe_ratio if pe_ratio else 'N/A'}}
- Profit Margin: {{profit_margin:.2f}}%
- Dividend Yield: {{dividend_yield:.2f}}%
- Beta: {{beta if beta else 'N/A'}}

KEY OBSERVATIONS:
{{chr(10).join(f"• {{factor}}" for factor in fundamental_factors)}}



ANALYST SUMMARY:
Based on technical and fundamental analysis, {{symbol}} shows {{price_trend.lower()}} with {{risk_level.lower()}} volatility profile. 
The analysis reflects current market conditions and financial performance metrics for informational purposes.

DISCLAIMER: This analysis is for informational purposes only and does not constitute investment advice.
"""
    
    return analysis_report

@tool
def generate_stock_report(stock_data: str, analysis_data: str) -> str:
    """
    Generate a comprehensive stock report based on gathered data and analysis.
    Creates a professional PDF report and uploads to S3 for documentation purposes.
    
    Args:
        stock_data: Raw stock data from the data gathering agent
        analysis_data: Analysis results from the performance analyzer
    
    Returns:
        Report generation summary with PDF creation and S3 upload status
    """
    import re
    
    # Extract key information for report
    symbol_match = re.search(r'Stock Symbol: ([A-Z]+)', stock_data)
    price_match = re.search(r'Current Price: \\$([\\d.]+)', stock_data)
    company_match = re.search(r'Company Name: ([^\\n]+)', stock_data)
    sector_match = re.search(r'Sector: ([^\\n]+)', stock_data)
    ytd_match = re.search(r'YTD Performance: ([\\d.-]+)%', analysis_data)
    risk_match = re.search(r'Volatility Risk: ([A-Z]+)', analysis_data)
    
    symbol = symbol_match.group(1) if symbol_match else 'UNKNOWN'
    current_price = float(price_match.group(1)) if price_match else 0
    company_name = company_match.group(1).strip() if company_match else 'N/A'
    sector = sector_match.group(1).strip() if sector_match else 'N/A'
    ytd_performance = float(ytd_match.group(1)) if ytd_match else 0
    risk_level = risk_match.group(1) if risk_match else 'MEDIUM'
    
    # Generate PDF report and upload to S3
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_filename = f"{{symbol}}_Stock_Report_{{timestamp}}.pdf"
    
    try:
        s3_path = create_and_upload_stock_report_pdf(
            symbol, company_name, sector, current_price, 
            ytd_performance, risk_level, stock_data, analysis_data, pdf_filename
        )
        pdf_status = f"PDF report uploaded to S3: {{s3_path}}"
    except Exception as e:
        pdf_status = f"PDF generation/upload failed: {{str(e)}}"
    
    report_summary = f"""STOCK REPORT GENERATION:
===============================
Stock: {{symbol}} ({{company_name}})
Sector: {{sector}}
Current Price: ${{current_price:.2f}}

REPORT SUMMARY:
- Technical Analysis: {{ytd_performance:.2f}}% YTD performance
- Risk Assessment: {{risk_level}} volatility risk
- Report Type: Comprehensive stock analysis for informational purposes
- Generated: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}

{{pdf_status}}

REPORT CONTENTS:
• Executive Summary with key metrics
• Detailed market data and financial metrics
• Technical and fundamental analysis
• Risk assessment and observations
• Professional formatting for documentation

DISCLAIMER: This report is for informational and educational purposes only. 
It does not constitute investment advice or recommendations.
"""
    
    return report_summary

def create_and_upload_stock_report_pdf(symbol, company_name, sector, price, ytd_perf, risk_level, stock_data, analysis_data, filename):
    """Create a professional PDF stock report and upload to S3"""
    
    # Create PDF in temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
        doc = SimpleDocTemplate(tmp_file.name, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []
        
        # Title
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=18,
            spaceAfter=30,
            textColor=colors.darkblue
        )
        story.append(Paragraph(f"Stock Analysis Report: {{symbol}}", title_style))
        story.append(Spacer(1, 12))
        
        # Executive Summary
        story.append(Paragraph("Executive Summary", styles['Heading2']))
        summary_data = [
            ['Metric', 'Value'],
            ['Stock Symbol', symbol],
            ['Company Name', company_name],
            ['Sector', sector],
            ['Current Price', f"${{price:.2f}}"],
            ['YTD Performance', f"{{ytd_perf:.2f}}%"],
            ['Risk Level', risk_level]
        ]
        
        summary_table = Table(summary_data)
        summary_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 12),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black)
        ]))
        
        story.append(summary_table)
        story.append(Spacer(1, 20))
        
        # Stock Data Section
        story.append(Paragraph("Market Data", styles['Heading2']))
        story.append(Paragraph(stock_data.replace('\\n', '<br/>'), styles['Normal']))
        story.append(Spacer(1, 20))
        
        # Analysis Section
        story.append(Paragraph("Performance Analysis", styles['Heading2']))
        story.append(Paragraph(analysis_data.replace('\\n', '<br/>'), styles['Normal']))
        
        # Generate timestamp
        story.append(Spacer(1, 20))
        story.append(Paragraph(f"Report Generated: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}", styles['Normal']))
        story.append(Paragraph("This report is for informational purposes only.", styles['Normal']))
        
        doc.build(story)
        
        # Upload to S3
        s3_key = datetime.now().strftime('%Y/%m/%d') + "/" + filename
        
        try:
            s3_client.upload_file(tmp_file.name, S3_BUCKET_NAME, s3_key)
            s3_path = f"s3://{{S3_BUCKET_NAME}}/{{s3_key}}"
            
            # Clean up temporary file
            os.unlink(tmp_file.name)
            
            return s3_path
            
        except Exception as e:
            # Clean up temporary file on error
            os.unlink(tmp_file.name)
            raise e

# Custom wrapper to make SagemakerEndpoint work with LangGraph tool binding
class SagemakerLLMWrapper:
    def __init__(self, sagemaker_llm, tools):
        self.sagemaker_llm = sagemaker_llm
        self.tools = tools
        self.tool_map = {{tool.name: tool for tool in tools}}
    
    def bind_tools(self, tools):
        # Return self since we're already configured with tools
        return self
    
    def invoke(self, messages):
        # Extract the user message content
        user_content = ""
        for msg in messages:
            if isinstance(msg, HumanMessage):
                user_content = msg.content
                break
        
        # Check if this is a stock analysis request
        if any(keyword in user_content.lower() for keyword in ['analyze', 'stock', 'ticker', 'symbol']):
            # Extract stock symbol from user input
            stock_match = re.search(r'\\b([A-Z]{{2,5}})\\b', user_content.upper())
            if stock_match:
                stock_symbol = stock_match.group(1)
                
                # Step 1: Gather stock data
                print(f"Step 1: Gathering data for {{stock_symbol}}...")
                stock_data = self.tools[0].invoke({{"stock_symbol": stock_symbol}})
                
                # Step 2: Analyze stock performance
                print(f"Step 2: Analyzing {{stock_symbol}} performance...")
                analysis_result = self.tools[1].invoke({{"stock_data": stock_data}})
                
                # Step 3: Generate stock report
                print(f"Step 3: Generating report for {{stock_symbol}}...")
                report_result = self.tools[2].invoke({{"stock_data": stock_data, "analysis_data": analysis_result}})
                
                # Return comprehensive response
                full_response = f"""**COMPREHENSIVE STOCK ANALYSIS REPORT**

**Step 1 - Stock Data Gathering:**
{{stock_data}}

**Step 2 - Performance Analysis:**
{{analysis_result}}

**Step 3 - Report Generation:**
{{report_result}}

---
**ANALYSIS COMPLETE:** Comprehensive stock analysis has been performed and a detailed PDF report has been generated and uploaded to S3 for documentation purposes."""
                
                return AIMessage(content=full_response)
            else:
                return AIMessage(content="Please provide a valid stock symbol (e.g., AAPL, GOOGL, TSLA) for analysis.")
        
        # For other messages, use the SageMaker model normally
        system_msg = """You are a professional stock analyst. Provide helpful responses about stock analysis, market trends, and financial metrics for informational purposes only."""
        
        full_prompt = f"{{system_msg}}\\n\\nUser: {{user_content}}"
        
        # Get response from SageMaker endpoint
        response = self.sagemaker_llm.invoke(full_prompt)
        
        # Return a proper LangChain AIMessage
        return AIMessage(content=response)

# Define the agent using SageMaker endpoint
def create_agent():
    """Create and configure the LangGraph stock analysis agent with SageMaker endpoint"""
    
    # Your SageMaker endpoint configuration
    endpoint_name = "{endpoint_name}"
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            # GPT-OSS harmony format payload structure
            payload = {{
                "model": "/opt/ml/model",
                "input": [
                    {{
                        "role": "system",
                        "content": "You are a professional stock analyst. Analyze stocks and provide detailed information for educational purposes only, without investment recommendations."
                    }},
                    {{
                        "role": "user",
                        "content": prompt
                    }}
                ],
                "max_output_tokens": model_kwargs.get("max_new_tokens", 2048),
                "stream": "false",
                "temperature": model_kwargs.get("temperature", 0.1),
                "top_p": model_kwargs.get("top_p", 1)
            }}
            input_str = json.dumps(payload)
            return input_str.encode("utf-8")

        def transform_output(self, output: bytes) -> str:
            # Parse harmony format response
            decoded_output = output.read().decode("utf-8")
            response_json = json.loads(decoded_output)
            
            if 'output' in response_json and isinstance(response_json['output'], list):
                for item in response_json['output']:
                    if item.get('type') == 'message' and item.get('role') == 'assistant':
                        content = item.get('content', [])
                        for content_item in content:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
                
                # Fallback parsing for different harmony format structures
                for item in response_json['output']:
                    if item.get('type') != 'reasoning' and 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if content_item.get('type') == 'output_text' and 'text' in content_item:
                                return content_item['text']
                
                for item in response_json['output']:
                    if 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if 'text' in content_item:
                                return content_item['text']
            
            return str(response_json)

    # Initialize SageMaker LLM with harmony format
    content_handler = ContentHandler()
    sagemaker_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-2",
        model_kwargs={{
            "max_new_tokens": 2048, 
            "do_sample": True, 
            "temperature": 0.1,  # Lower temperature for consistent analysis
            "top_p": 1
        }},
        content_handler=content_handler
    )
    
    # Create tools (3 tools: data gathering, analysis, report generation)
    tools = [gather_stock_data, analyze_stock_performance, generate_stock_report]
    
    # Wrap SageMaker LLM to work with LangGraph
    llm_with_tools = SagemakerLLMWrapper(sagemaker_llm, tools)
    
    # System message for stock analysis
    system_message = """You are a professional stock analyst with expertise in technical analysis, fundamental analysis, and report generation. 

Your role is to:
1. Gather comprehensive stock data from multiple sources including price history, financial metrics, and market data
2. Analyze stock performance using both technical and fundamental analysis techniques
3. Generate professional stock reports for documentation and educational purposes

Provide informational analysis only, without investment recommendations or advice."""
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {{"messages": [response]}}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_stock_sagemaker(payload):
    """
    Invoke the stock analysis agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({{"messages": [HumanMessage(content=user_input)]}})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()
'''
    
    # Write the file
    with open(filename, 'w') as f:
        f.write(code_content)
    
    print(f"Created {filename} with endpoint: {endpoint_name} and bucket: {bucket_name}")
    return filename

# Example usage
if __name__ == "__main__":
    # Replace with your actual values
    sagemaker_endpoint_name = "jumpstart-dft-openai-reasoning-gpt-20250805-192527"
    bucket_name = "surya-495365983931"
    create_agentcore_deployment_file(sagemaker_endpoint_name, bucket_name)


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating IAM Role for Bedrock AgentCore Runtime

Before deploying our Stock Analysis agent to Bedrock AgentCore Runtime, we need to create an IAM role with the appropriate permissions. This role will allow AgentCore to:

• **Invoke your SageMaker endpoint** for GPT-OSS model inference  
• **Manage ECR repositories** for storing container images  
• **Write CloudWatch logs** for monitoring and debugging  
• **Access Bedrock AgentCore workload services** for runtime operations  
• **Send telemetry data** to X-Ray and CloudWatch for observability  

The function below creates a comprehensive IAM role with five custom policies plus the AWS managed 
AmazonBedrockFullAccess policy. Each policy is scoped to only the resources needed for AgentCore operations, 
following the principle of least privilege.

**Key Features**:  

• **Automatic policy creation** with timestamped names to avoid conflicts  
• **Error handling** for existing roles and policies  
• **Resource-specific permissions** (scoped to your SageMaker endpoint and ECR repositories)  
• **Ready-to-use role ARN** for AgentCore configuration  

This approach ensures your agent has exactly the permissions it needs to run securely in the AgentCore Runtime 
environment while maintaining access to your specific SageMaker model endpoint.


In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

def add_s3_inline_policy_to_role(role_name, s3_bucket_name, region="us-west-2"):
    """
    Add S3 permissions as an inline policy to an existing role
    
    Args:
        role_name: Existing IAM role name
        s3_bucket_name: S3 bucket name for PDF uploads
        region: AWS region
    """
    
    iam_client = boto3.client('iam', region_name=region)
    
    print(f"Adding S3 inline policy to role: {role_name}")
    print(f"S3 Bucket: {s3_bucket_name}")
    
    # S3 policy document
    s3_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:PutObject",
                    "s3:PutObjectAcl",
                    "s3:GetObject",
                    "s3:DeleteObject"
                ],
                "Resource": f"arn:aws:s3:::{s3_bucket_name}/*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "s3:ListBucket",
                    "s3:GetBucketLocation"
                ],
                "Resource": f"arn:aws:s3:::{s3_bucket_name}"
            }
        ]
    }
    
    # Add inline policy to role
    policy_name = "S3AccessForStockAnalysis"
    
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=policy_name,
            PolicyDocument=json.dumps(s3_policy)
        )
        print(f"Successfully added inline S3 policy: {policy_name}")
        print(f"Role {role_name} now has access to bucket: {s3_bucket_name}")
        
    except ClientError as e:
        print(f"Error adding inline policy: {e}")
        raise
    
    return True

def list_role_policies(role_name, region="us-west-2"):
    """List all policies attached to a role"""
    
    iam_client = boto3.client('iam', region_name=region)
    
    print(f"Policies for role: {role_name}")
    print("-" * 50)
    
    try:
        # List attached managed policies
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
        print(f"Managed Policies ({len(attached_policies['AttachedPolicies'])}):")
        for policy in attached_policies['AttachedPolicies']:
            print(f"  - {policy['PolicyName']}")
        
        # List inline policies
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        print(f"\nInline Policies ({len(inline_policies['PolicyNames'])}):")
        for policy_name in inline_policies['PolicyNames']:
            print(f"  - {policy_name}")
            
    except ClientError as e:
        print(f"Error listing policies: {e}")

if __name__ == "__main__":
    role_name = "MyBedrockAgentCoreRole"
    s3_bucket_name = "surya-495365983931"
    region = "us-west-2"
    
    # First, let's see what policies are currently attached
    print("Current policies:")
    list_role_policies(role_name, region)
    
    print("\n" + "="*60)
    
    # Add S3 permissions as inline policy
    add_s3_inline_policy_to_role(role_name, s3_bucket_name, region)
    
    print("\n" + "="*60)
    
    # Show updated policies
    print("Updated policies:")
    list_role_policies(role_name, region)


### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
agent_name = "langgraph_stock_analyzer_agent"

boto_session = Session()
region = "us-west-2"

agentcore_runtime = Runtime()

# Configure the agent (this doesn't require Docker)
response = agentcore_runtime.configure(
    entrypoint="langgraph_stock_sagemaker_gpt_oss.py",
    auto_create_execution_role=False,
    execution_role=role_arn,  # Use custom role
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime


In [ ]:
launch_result = agentcore_runtime.launch(use_codebuild=True)

In [ ]:
agent_arn = launch_result.agent_arn
print(f"Agent_ARN='{agent_arn}'")

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Analyze AAPL stock for investment"})

invoke_response

### Parsing and Displaying Stock Analysis Results

After invoking our stock analysis agent through AgentCore Runtime, we need to parse and format the response 
for clear presentation. The agent returns a comprehensive analysis in JSON format containing three distinct 
phases of the investment research process.

This parsing function extracts and displays the stock analysis in a structured format:

**Response Processing**:  

• **Decodes the byte stream** from AgentCore into readable text  
• **Parses the JSON response** containing the complete stock analysis  
• **Extracts three main sections** using regex pattern matching:  

    • Step 1: Stock data gathering with market metrics and company information  
    • Step 2: Performance analysis with technical indicators and fundamental evaluation  
    • Step 3: Investment decision with buy/sell/hold recommendations and PDF report generation  

**Key Information Extraction**:  

• Investment decision (INVEST/HOLD/AVOID) with confidence levels and visual indicators  
• Investment rating (STRONG BUY/BUY/HOLD/SELL) showing the quantitative assessment  
• Financial metrics including current price, P/E ratio, market cap, and YTD performance  
• Risk assessment with volatility analysis and portfolio allocation guidance  
• PDF report status with S3 storage location or error details  

**Error Handling**:  

• Gracefully handles JSON parsing errors  
• Falls back to plain text display if structured parsing fails  
• Provides debugging information for troubleshooting  
• Handles PDF generation failures with detailed error messages  

**Additional Features**:

• Programmatic data access for integration with other systems  
• Structured metrics extraction for quantitative analysis  
• Investment summary generation for executive reporting  
• S3 path validation for document retrieval  

This formatted output makes it easy to review the agent's investment analysis process, access key financial metrics programmatically, and present professional stock research results to portfolio managers and investment committees. 

The parsing function also provides direct access to generated PDF reports stored in Amazon S3 for comprehensive documentation and audit trails.

In [ ]:
import json
import re

def parse_bedrock_agentcore_stock_response(invoke_response):
    """Parse the complete Bedrock AgentCore stock analysis response from byte chunks"""
    
    # Combine all byte chunks into one string
    response_chunks = invoke_response['response']
    complete_response = b''.join(response_chunks).decode('utf-8')
    
    try:
        # Parse the JSON (it's a JSON string containing the analysis)
        data = json.loads(complete_response)
        
        print("COMPREHENSIVE STOCK ANALYSIS REPORT")
        print("=" * 80)
        
        # Extract the three main sections using regex
        step1_match = re.search(r'\*\*Step 1 - Stock Data Gathering:\*\*(.*?)\*\*Step 2', data, re.DOTALL)
        step2_match = re.search(r'\*\*Step 2 - Performance Analysis:\*\*(.*?)\*\*Step 3', data, re.DOTALL)
        step3_match = re.search(r'\*\*Step 3 - Report Generation:\*\*(.*?)---', data, re.DOTALL)
        
        print(f"\nSTEP 1 - STOCK DATA GATHERING:")
        print("-" * 50)
        if step1_match:
            data_section = step1_match.group(1).strip()
            print(data_section)
        else:
            print("Could not extract stock data section")
        
        print(f"\nSTEP 2 - PERFORMANCE ANALYSIS:")
        print("-" * 50)
        if step2_match:
            analysis_section = step2_match.group(1).strip()
            print(analysis_section)
        else:
            print("Could not extract analysis section")
        
        print(f"\nSTEP 3 - REPORT GENERATION:")
        print("-" * 50)
        if step3_match:
            report_section = step3_match.group(1).strip()
            print(report_section)
            
            # Extract key report generation info
            ytd_perf_match = re.search(r'Technical Analysis: ([^\n]+)', report_section)
            risk_assess_match = re.search(r'Risk Assessment: ([^\n]+)', report_section)
            report_type_match = re.search(r'Report Type: ([^\n]+)', report_section)
            generated_match = re.search(r'Generated: ([^\n]+)', report_section)
            
            print(f"\nKEY REPORT METRICS:")
            print("-" * 30)
            if ytd_perf_match:
                print(f"YTD Performance: {ytd_perf_match.group(1).strip()}")
            if risk_assess_match:
                print(f"Risk Assessment: {risk_assess_match.group(1).strip()}")
            if report_type_match:
                print(f"Report Type: {report_type_match.group(1).strip()}")
            if generated_match:
                print(f"Generated: {generated_match.group(1).strip()}")
        else:
            print("Could not extract report section")
        
        # Extract stock summary from the data gathering section
        symbol_match = re.search(r'Stock Symbol: ([^\n]+)', data)
        company_match = re.search(r'Company Name: ([^\n]+)', data)
        current_price_match = re.search(r'Current Price: \$([^\n]+)', data)
        ytd_return_match = re.search(r'YTD Return: ([^\n]+)', data)
        market_cap_match = re.search(r'Market Cap: \$([^\n]+)', data)
        pe_ratio_match = re.search(r'P/E Ratio: ([^\n]+)', data)
        
        if symbol_match:
            print(f"\nSTOCK SUMMARY:")
            print("-" * 25)
            if symbol_match:
                print(f"Symbol: {symbol_match.group(1).strip()}")
            if company_match:
                print(f"Company: {company_match.group(1).strip()}")
            if current_price_match:
                print(f"Current Price: ${current_price_match.group(1).strip()}")
            if market_cap_match:
                print(f"Market Cap: ${market_cap_match.group(1).strip()}")
            if pe_ratio_match:
                print(f"P/E Ratio: {pe_ratio_match.group(1).strip()}")
            if ytd_return_match:
                print(f"YTD Return: {ytd_return_match.group(1).strip()}")
        
        # Check for PDF upload status
        pdf_match = re.search(r'PDF.*?(?:uploaded to S3: (s3://[^\n]+)|generation/upload failed: ([^\n]+))', data, re.IGNORECASE)
        if pdf_match:
            print(f"\nPDF REPORT STATUS:")
            print("-" * 25)
            if pdf_match.group(1):  # Successful upload
                print(f"PDF uploaded to: {pdf_match.group(1)}")
            elif pdf_match.group(2):  # Failed upload
                print(f"PDF upload failed: {pdf_match.group(2)}")
        
        print(f"\nSTOCK ANALYSIS COMPLETE")
        print("=" * 80)
        
        return {
            'stock_data_gathering': step1_match.group(1).strip() if step1_match else None,
            'performance_analysis': step2_match.group(1).strip() if step2_match else None,
            'report_generation': step3_match.group(1).strip() if step3_match else None,
            'stock_symbol': symbol_match.group(1).strip() if symbol_match else None,
            'company_name': company_match.group(1).strip() if company_match else None,
            'current_price': current_price_match.group(1).strip() if current_price_match else None,
            'ytd_performance': ytd_perf_match.group(1).strip() if ytd_perf_match else None,
            'risk_assessment': risk_assess_match.group(1).strip() if risk_assess_match else None,
            'report_type': report_type_match.group(1).strip() if report_type_match else None,
            'generated_time': generated_match.group(1).strip() if generated_match else None,
            'ytd_return': ytd_return_match.group(1).strip() if ytd_return_match else None,
            'pe_ratio': pe_ratio_match.group(1).strip() if pe_ratio_match else None,
            'market_cap': market_cap_match.group(1).strip() if market_cap_match else None,
            'pdf_status': pdf_match.group(1) if pdf_match and pdf_match.group(1) else pdf_match.group(2) if pdf_match and pdf_match.group(2) else None,
            'raw_response': data
        }
        
    except json.JSONDecodeError as e:
        print(f"JSON Error: {e}")
        print(f"Raw response length: {len(complete_response)}")
        print(f"First 500 chars: {complete_response[:500]}")
        
        # Try to parse as plain text if JSON fails
        print("\nATTEMPTING PLAIN TEXT PARSING:")
        print("-" * 40)
        print(complete_response)
        return {'raw_response': complete_response}

# Parse your existing response
stock_analysis = parse_bedrock_agentcore_stock_response(invoke_response)

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
import json

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Analyze APPL Stock"})
)

# Use the parsing function we created earlier
stock_analysis = parse_bedrock_agentcore_stock_response(boto3_response)

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!